In [69]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import os
from IPython.display import display
import plotly.io as pio
pio.renderers.default = 'notebook'

In [46]:
dfo = pd.read_csv('processed_emissions.csv')

dfo['Country'] = dfo['Country'].fillna(method='ffill')

if dfo['Country'].notna().iloc[0] != 'European Union':
    dfo['Country'] = dfo['Country'].apply(lambda x: 'European Union' 
                                          if x == df['Country'].notna().iloc[0] else x)


dfo['Country'] = dfo['Country'].fillna(method='ffill')
#df.to_csv('processed_emissions_modified.csv', index=False)

In [47]:
d2 = pd.read_csv("processed_emissions_modified.csv")
unique_emission_types = d2['Type of Emission'].unique()
country = d2['Country'].unique()

print(unique_emission_types)


[nan '    Emissions' '        CO2 emissions (MMtonnes CO2)'
 '            Coal and coke (MMtonnes CO2)'
 '            Consumed natural gas (MMtonnes CO2)'
 '            Petroleum and other liquids (MMtonnes CO2)']


In [123]:
df = pd.read_csv("processed_emissions_modified.csv") 
filtered_df = df[
    (df['Type of Emission'] == '        CO2 emissions (MMtonnes CO2)') &
    (df['Country'] == 'European Union')
]

years = [str(year) for year in range(1980, 2020)]

filtered_df_years = filtered_df[years]

filtered_df_years = filtered_df_years.apply(pd.to_numeric, errors='coerce')

emissions_by_year = filtered_df_years.sum()

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=emissions_by_year.index,
    y=emissions_by_year.values,
    mode='lines+markers',
    name='CO2 Emissions',
    line=dict(color='blue', width=2),
    marker=dict(size=6, color='red')
))

fig.update_layout(
    title='Evolution of CO2 Emissions for the European Union (1980-2019)',
    xaxis_title='Year',
    yaxis_title='CO2 Emissions (MMtonnes CO2)',
    xaxis=dict(
        tickmode='linear',
        tickvals=list(range(1980, 2020)),
        ticktext=[str(year) for year in range(1980, 2020)],
        tickangle=-45
    ),
    yaxis=dict(
        title='CO2 Emissions (MMtonnes CO2)',
        rangemode='tozero'
    ),
    plot_bgcolor='white',
    paper_bgcolor='lightgray'
)

fig.show()

In [118]:
co2_df = d2[d2['Type of Emission'].str.strip() == 'CO2 emissions (MMtonnes CO2)']

years = [str(year) for year in range(1980, 2023)]

co2_df.loc[:, years] = co2_df[years].apply(pd.to_numeric, errors='coerce')

co2_df = co2_df.drop_duplicates(subset=['Country'])

co2_df.set_index('Country', inplace=True)
total_emissions = co2_df[years].sum(axis=1)

sorted_emissions = total_emissions.sort_values(ascending=False)

top_20 = sorted_emissions.head(20)
bottom_20 = sorted_emissions.tail(20)

middle_index = len(sorted_emissions) // 2
if len(sorted_emissions) < 20:
    middle_20 = sorted_emissions
else:
    middle_20 = sorted_emissions.iloc[middle_index - 5: middle_index + 5]

print("Top 20 countries with highest CO2 emissions:")
print(top_20)
print("\nBottom 20 countries with lowest CO2 emissions:")
print(bottom_20)
print("\nMiddle 20 countries with CO2 emissions:")
print(middle_20)

Top 20 countries with highest CO2 emissions:
Country
Non-OPEC                   1099336.572561
Non-OECD                    621492.401469
OECD                        534792.262455
OECD - North America        264033.009115
China                       244316.443166
United States               226226.987116
OECD - Europe               179442.282205
European Union              151951.378569
OECD - Asia And Oceania      83238.135426
OPEC                         56948.091363
Russia                       52309.139521
India                        50212.609054
Japan                        48533.939993
Persian Gulf                 46750.484226
Former U.S.S.R.              45510.782744
Germany                      26740.573594
United Kingdom               23047.705693
Canada                       21925.366524
South Korea                  19106.821483
Italy                        17110.155212
dtype: object

Bottom 20 countries with lowest CO2 emissions:
Country
Antarctica                    6.30343

In [127]:
file_path = "average-latitude-longitude-countries.csv"
dfA = pd.read_csv(file_path)

def categorize_latitude(latitude):
    if latitude > 23.5:
        return "North of Tropic of Cancer"
    elif 0 < latitude <= 23.5:
        return "Tropic of Cancer"
    elif latitude == 0:
        return "Equator"
    elif -23.5 <= latitude < 0:
        return "Tropic of Capricorn"
    else:
        return "South of Tropic of Capricorn"

dfA['Category'] = dfA['Latitude'].apply(categorize_latitude)

grouped = dfA.groupby('Category')['Country'].apply(list)

for category, countries in grouped.items():
    print(f"Category: {category}")
    print(f"Number of countries: {len(countries)}")
    print("Countries:", ", ".join(countries))
    print("\n")

Category: Equator
Number of countries: 1
Countries: Congo, The Democratic Republic of the


Category: North of Tropic of Cancer
Number of countries: 93
Countries: Andorra, United Arab Emirates, Afghanistan, Albania, Armenia, Asia/Pacific Region, Austria, Azerbaijan, Bosnia and Herzegovina, Bangladesh, Belgium, Bulgaria, Bahrain, Bermuda, Bahamas, Bhutan, Belarus, Canada, Switzerland, China, Cyprus, Czech Republic, Germany, Denmark, Algeria, Estonia, Egypt, Western Sahara, Spain, Europe, Finland, Faroe Islands, France, United Kingdom, Georgia, Gibraltar, Greenland, Greece, Croatia, Hungary, Ireland, Israel, Iraq, Iran, Islamic Republic of, Iceland, Italy, Jordan, Japan, Kyrgyzstan, Korea, Democratic People's Republic of, Korea, Republic of, Kuwait, Kazakhstan, Lebanon, Liechtenstein, Lithuania, Luxembourg, Latvia, Libyan Arab Jamahiriya, Morocco, Monaco, Moldova, Republic of, Montenegro, Macedonia, Mongolia, Malta, Netherlands, Norway, Nepal, Pakistan, Poland, Saint Pierre and Miquelon,

In [128]:
years = [str(year) for year in range(1980, 2013)]  # Years as strings from "1980" to "2012"

df_copy = df.copy()

df_copy[years] = df_copy[years].apply(pd.to_numeric, errors='coerce')

emission_types = ['        CO2 emissions (MMtonnes CO2)',
    '            Coal and coke (MMtonnes CO2)',
    '            Consumed natural gas (MMtonnes CO2)',
    '            Petroleum and other liquids (MMtonnes CO2)'
]

df_copy['Stock Emissions (1980-2012)'] = pd.NA

for emission_type in emission_types:
    mask = df_copy['Type of Emission'] == emission_type
    df_copy.loc[mask, 'Stock Emissions (1980-2012)'] = df_copy.loc[mask, years].sum(axis=1)

#remaining_mask = ~df_copy['Type of Emission'].isin(emission_types)
#df_copy.loc[remaining_mask, 'Stock Emissions (1980-2012)'] = df_copy.loc[remaining_mask, years].sum(axis=1)

df['Stock Emissions (1980-2012)'] = df_copy['Stock Emissions (1980-2012)']

df.head(20)


,Country,Type of Emission,1949,1950,1951,1952,1953,1954,1955,1956,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,Stock Emissions (1980-2012)
0,European Union,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
1,European Union,Emissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
2,European Union,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3095.50335549,3151.795757334,3158.350350425,3229.106490498,3173.837904551,3050.515706002,2674.174407197,2861.862122568,2810.895446026,121511.442133
3,European Union,Coal and coke (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,928.0586723010746,924.3504438731959,881.395946525172,885.8347672131258,837.4372789408834,686.1986177198614,553.60432220144,625.2556464438163,638.1503754676833,44943.461508
4,European Union,Consumed natural gas (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,665.44437301,693.833401501,733.071018948,771.853227747,760.421619714,785.818991333,750.968713961,788.586325812,672.944252201,21430.969333
5,European Union,Petroleum and other liquids (MMton...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1502.0003101813543,1533.6119119599196,1543.883384950255,1571.418495536,1575.979005895,1578.4980969494302,1369.6013710344841,1448.0201503162598,1499.8008183615273,55137.011292
6,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
7,Austria,Emissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>
8,Austria,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,61.136785464,63.186494439,63.750893511,66.333644577,63.769949626,65.360858771,57.76380755,60.910858543,57.876438884,1990.777655
9,Austria,Coal and coke (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.205085239638642,11.967926710380212,11.230113679329772,11.864013919872137,10.384165498079478,10.507913733946198,9.328810357978634,9.511334410119511,9.48316607267243,424.510927


In [129]:
filtered_df = df[df['Type of Emission'].isin(emission_types)].copy() 

years = [str(year) for year in range(2013, 2023)] 
for year in years:
    filtered_df[year] = pd.to_numeric(filtered_df[year], errors='coerce')
filtered_df['Stock Emissions (1980-2012)'] = pd.to_numeric(filtered_df['Stock Emissions (1980-2012)'], 
                                                           errors='coerce')


for year in years:
    change_col = f'% change of {year}'
    
    filtered_df.loc[:, change_col] = (filtered_df[year] / filtered_df['Stock Emissions (1980-2012)']) * 100

#filtered_df.to_csv('updated_data.csv', index=False)

filtered_df.head(30)

,Country,Type of Emission,1949,1950,1951,1952,1953,1954,1955,1956,...,% change of 2013,% change of 2014,% change of 2015,% change of 2016,% change of 2017,% change of 2018,% change of 2019,% change of 2020,% change of 2021,% change of 2022
2,European Union,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.661391,2.547499,2.593826,2.599221,2.657451,2.611966,2.510476,2.200759,2.355220,2.313276
3,European Union,Coal and coke (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.149765,2.064947,2.056696,1.961122,1.970998,1.863313,1.526804,1.231779,1.391205,1.419896
4,European Union,Consumed natural gas (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.494066,3.105060,3.237527,3.420615,3.601579,3.548237,3.666745,3.504129,3.679658,3.140055
5,European Union,Petroleum and other liquids (MMton...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.754781,2.724124,2.781456,2.800085,2.850025,2.858296,2.862865,2.483996,2.626222,2.720134
8,Austria,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.236763,3.071000,3.173960,3.202311,3.332047,3.203268,3.283182,2.901570,3.059652,2.907228
9,Austria,Coal and coke (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.828761,2.639528,2.819227,2.645424,2.794749,2.446148,2.475299,2.197543,2.240539,2.233904
10,Austria,Consumed natural gas (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.554293,3.244353,3.461975,3.615378,3.924310,3.656087,3.818521,3.630636,3.844452,3.364721
11,Austria,Petroleum and other liquids (MMton...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.260045,3.164147,3.189165,3.242667,3.289260,3.304018,3.368684,2.865325,3.044246,2.973713
14,Belgium,CO2 emissions (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.863420,2.659139,2.799021,2.832514,2.896827,3.096630,3.005918,2.660354,2.859883,2.714450
15,Belgium,Coal and coke (MMtonnes CO2),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.269449,1.138139,1.063166,1.001245,0.972673,0.990104,1.037585,0.800876,0.886137,0.822204
